In [48]:
from Modules.EventusParser import parser, eventus_formatter
from Modules.Misc import skeleton
from scipy import stats
from tqdm.auto import tqdm
import pandas as pd
import re

tqdm.pandas()
# Set the display option to show all columns
pd.set_option('display.max_columns', None, 'display.max_colwidth', None)
output = './Data/Output/'
event_date = '20220516'

# 1. Eventus average announcement returns

In [57]:
#   767 firms which filed 8-K with 5.07 with HQ in California
ca = pd.read_csv('./Data/Input/Misc/CA_sample_2022_repeal.csv')

#   Linking table
#   NOTE:   I didn't use the cusip-permno linking table as it lost 70% of firms (cusip6 merge)
cik_cusip = pd.read_csv('./Data/Input/Misc/cik-cusip-maps.csv')

#   Get all possible matches first
ca_linked = ca.merge(cik_cusip, how='left', left_on='CIK', right_on='cik')
ca_linked['date'] = event_date
ca_linked['cusip_length'] = ca_linked['cusip8'].str.len()
#   EVENTUS requires 8 digit cusips, recall
#   -   first 6 digit identifies firm
#   -   the 7-8 digit identifies security
#   -   last digit is check digit
#   NOTE: There are some CIKs with multiple classes of shares
#   Hence, we value weight returns in eventus.
ca_linked.query('cusip_length >=8 and cusip8 == cusip8', inplace=True)

#   This is the required input for WRDS Eventus
#   NOTE:   The output from WRDS is text file which requires parsing
#   NOTE:   SAS or STATA can general data files
ca_linked[['cusip8', 'date']].to_csv(output + 'eventus_input_Q1.txt', sep=' ', index=False, header=False)

#   The eventus setting is 
#   -   value weighted returns
#   -   excluding dividends
#   -   market adjusted returns
#   -   Estimation period:
#           -   end -50 days before event
#           -   minimum estimation length 50
#           -   length 252 trading days
#           -   OLS
table = parser(
    open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q1.txt', 'r').read(),
    **eventus_formatter['Abnormal']
)

table

,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",769,3.86%,2.19%,501:268>>>,1.986,0.0235,12.699,<.0001,10.292,<.0001
1,"(-1,0)",769,2.24%,1.06%,452:317>>>,1.413,0.0788,7.533,<.0001,6.750,<.0001
2,"(0,+1)",769,1.24%,0.80%,435:334>>>,0.783,0.2167,5.681,<.0001,5.521,<.0001
3,"(-5,+5)",769,1.09%,0.66%,392:377>>,0.292,0.3850,1.994,0.0231,2.413,0.0079
4,"(-10,+10)",769,-2.26%,-2.09%,357:412,-0.440,0.3299,-4.592,<.0001,-0.117,0.4534
5,"(-1,+1)",769,2.50%,1.41%,443:326>>>,1.375,0.0846,8.195,<.0001,5.038,<.0001
6,"(-1,0)",769,1.61%,0.70%,406:363>>,1.081,0.1399,4.968,<.0001,2.368,0.0089
7,"(0,+1)",769,0.89%,0.60%,415:354>>,0.597,0.2753,4.267,<.0001,3.018,0.0013
8,"(-5,+5)",769,4.16%,2.48%,454:315>>>,1.193,0.1164,7.527,<.0001,5.832,<.0001
9,"(-10,+10)",769,1.00%,-0.12%,388:381,0.207,0.4179,-0.271,0.3931,1.070,0.1424


# 2. Voting outcomes of new female directors

In [50]:
#   ISS VOTING DATA 
iss = pd.read_csv(
            './Data/Input/ISS/voting.csv',
            usecols=[
                'Name', 'sponsor', 'AgendaGeneralDesc', 'voteResult', 'MeetingDate', 'CompanyID', 'ItemDesc',
                'votedFor', 'votedAgainst', 'votedAbstain', 'VotedWithheld', 'CUSIP'
            ]
        )\
        .query('sponsor == "Management" and AgendaGeneralDesc == "Elect Director" and voteResult in ["Pass", "Fail"]')\
        .query('"2022-02-01" <= MeetingDate <= "2022-08-31"')\
        .rename({'CompanyID': 'company_id'}, axis=1)

#   We need a indicator for whether the firms are the identified CA firms
#   Firm level data, so CUSIP6 would be enough and also maximise matches
ca_ind = ca_linked[['cusip6', 'CIK']].assign(ca = 1)
iss['cusip6'] = iss['CUSIP'].str[:6]

#       merge in the indicator
iss = iss.merge(ca_ind, how = 'left', on = 'cusip6')

#   SANITY CHECK!
print(f'Voting data shape: {iss.shape} \n\t- CA firms {iss.query("ca==1")["company_id"].nunique()}')

#   CALCULATE SUPPORT 
#       first fill na for vote
filna = ['votedFor', 'votedAgainst', 'votedAbstain', 'VotedWithheld']
iss[filna] = iss[filna].fillna(0)
#       percentage of support for each director
iss.eval('support = votedFor / (votedFor + votedAgainst + votedAbstain + VotedWithheld)', inplace=True)
#       to calculate excess support we need to know
#       1.  number of directors on the board
#       2.  the average support for all the directors
iss = iss.assign(
    n_directors = iss.groupby(['MeetingDate', 'company_id'])['support'].transform('size'),
    tot_support = iss.groupby(['MeetingDate', 'company_id'])['support'].transform('sum'),
)
#       Excess Support
iss.eval('excess_support = support - (tot_support - support)/(n_directors - 1)', inplace = True)

#   FEMALE INDICATOR
#       extract full name from the voting data
iss['FULLNAME'] = iss['ItemDesc'].str.upper()\
                    .str.extract(
                        r'(?:(?:re)?elec[t]?(?:ion)?(?:trustee)?\s+|director\s+|appointment (?:of)?|re(?:new|appoint) |appoint |Co[-]?optation ){1,2}(.*?)(?: [.]?(?:as|by|to|class|conditional) .*?)?$', 
                        flags=re.IGNORECASE
                    ).replace(r'[^0-9a-zA-Z]', '')
iss['namekey'] = skeleton(iss['FULLNAME'])


Voting data shape: (24553, 15) 
	- CA firms 515


In [51]:
#       Use both ISS and board ex to maximise links
#       Remember to drop NA or there will be one to many links
iss_ind = pd.read_csv('./Data/Input/ISS/director.csv', usecols=['FULLNAME', 'Female'])\
                .drop_duplicates()\
                .dropna(subset=['Female'])
boardex = pd.read_csv('./Data/Input/BoardEX/individual.csv', usecols=['DirectorName', 'Gender', 'DirectorID'])\
                .rename({'Gender': 'Female'}, axis=1)\
                .drop_duplicates()\
                .dropna(subset=['Female'])
#       Standardise the dummy
iss_ind['Female'] = iss_ind['Female'].apply(lambda x: 1 if x == 'Yes' else 0)
boardex['Female'] = boardex['Female'].apply(lambda x: 1 if x == 'F' else 0)
#       Make sure names are in somewhat standardised format
iss_ind['namekey'] = skeleton(iss_ind['FULLNAME'])
boardex['namekey'] = skeleton(boardex['DirectorName'])
#       Concat the two together
name_f = pd.concat(
        [iss_ind, boardex], 
        ignore_index=True
#       here we are basically keeping records with dirID (if any)
    ).sort_values(['namekey', 'Female', 'DirectorID'])\
     .drop_duplicates(['namekey', 'Female'], keep='first')
#       there will be records where the same person is both a female and not
name_f = name_f[name_f['namekey'].duplicated() == False]
#    Merge in the female indicator
iss_gender = iss.merge(name_f, on='namekey', how='left')
iss_gender['Post'] = iss_gender['MeetingDate'].str.replace('-', '') > event_date

print(f'Post merge iss shape: {iss_gender.shape}')
print(f'Unique Name - Female links: {name_f.shape} ')
print(f'Observations with female dummy: {iss_gender[iss_gender["Female"].notna()].shape} (Female == 1, {iss_gender.query("Female==1").shape})')
print(f'Observations missing female dummy: {iss_gender[iss_gender["Female"].isna()].shape}')

print(f'Observations after {event_date}: {iss_gender[iss_gender["Post"] == True].shape}')
print(f'Observations before {event_date}: {iss_gender[iss_gender["Post"] == False].shape}')

Post merge iss shape: (24553, 26)
Unique Name - Female links: (808257, 5) 
Observations with female dummy: (16483, 26) (Female == 1, (5833, 26))
Observations missing female dummy: (8070, 26)
Observations after 20220516: (14954, 26)
Observations before 20220516: (9599, 26)


In [52]:
#   First time female directors 
#   We use the NOMINEE indicator
#       Mgmt Continuing    142296
#       No                  43099
#      *Mgmt NEW            16843
#       Shareholder           280
nominee = pd.read_csv('./Data/Input/ISS/director.csv', 
                usecols=['company_id', 'MeetingDate', 'FULLNAME', 'Nominee']
            )
nominee['namekey'] = skeleton(nominee['FULLNAME'])

iss_gender_new = iss_gender.merge(nominee, 
                on = ['company_id', 'MeetingDate', 'namekey'],
                how = 'left'
)
print(f'Observations with Nominee: {iss_gender_new[iss_gender_new["Nominee"].notna()].shape}')
print(f'Observations without Nominee: {iss_gender_new[iss_gender_new["Nominee"].isna()].shape}')

new_female = iss_gender_new.query('Female == 1 and Nominee == "Mgmt NEW"')

new_female_ca = new_female.query('ca==1')
new_female_nonca = new_female.query('ca!=1')

t_stat_support_rate, p_val_support_rate = stats.ttest_ind(
    new_female_ca.query('Post==1')['excess_support'], 
    new_female_ca.query('Post==0')['excess_support'], 
equal_var=False)

print(p_val_support_rate)

print(
    f'\nAverage Excess Votes (not CA, {new_female_nonca.shape}): \n',
    new_female_nonca.groupby('Post')['excess_support'].agg(['mean', 'size']),
    '\nAverage support (not CA): \n',
    new_female_nonca.groupby('Post')['support'].agg(['mean', 'size']),
    '\n\n',
    f'\nAverage Excess Votes (CA, {new_female_ca.shape}): \n',
    new_female_ca.groupby('Post')['excess_support'].agg(['mean', 'size']),
    '\nAverage support (CA): \n',
    new_female_ca.groupby('Post')['support'].agg(['mean', 'size'])
)

Observations with Nominee: (10784, 28)
Observations without Nominee: (13781, 28)
0.07982976561116017

Average Excess Votes (not CA, (416, 28)): 
            mean  size
Post                 
False  0.029417   182
True   0.022746   234 
Average support (not CA): 
            mean  size
Post                 
False  0.985023   182
True   0.967959   234 

 
Average Excess Votes (CA, (106, 28)): 
            mean  size
Post                 
False  0.013343    37
True   0.042647    69 
Average support (CA): 
            mean  size
Post                 
False  0.953881    37
True   0.965668    69


# 3. Reaction to new female directors pre/post

In [53]:
announce_path = './Data/Input/BoardEX/announcement.csv'
individu_path = './Data/Input/BoardEX/individual.csv'
company_path = './Data/Input/BoardEX/company.csv'

raw_announcement = pd.read_csv(announce_path)\
    .drop_duplicates()\
    .query('"2022-02-01" <= AnnouncementDate <= "2022-08-31"')\
    .merge(
        pd.read_csv(individu_path, usecols=['DirectorID', 'Gender']).drop_duplicates(),
        how = 'left',
        on = 'DirectorID'
    ).merge(
        pd.read_csv(company_path, usecols=['CIKCode', 'BoardID']).drop_duplicates(),
        how = 'left',
        left_on = 'CompanyID',
        right_on = 'BoardID'
    ).rename({'CIKCode':'CIK'}, axis=1)\
    
announcement = raw_announcement.query(f'CIK in [{",".join(ca["CIK"].astype(str))}] and Gender == "F"')
announcement = announcement[announcement['Description'].str.contains('will join')]

idf = announcement.query('RoleName == "Independent Director"')\
        .merge(ca_linked[['CIK', 'cusip8']], how='left', on='CIK')\
        .query('cusip8 == cusip8')
idf['AnnouncementDate'] = idf['AnnouncementDate'].str.replace('-', '')

idf_bef = idf.query(f'AnnouncementDate < "{event_date}"')
idf_aft = idf.query(f'AnnouncementDate >= "{event_date}"')

print('Before shape:', idf_bef.shape, '\nAfter shape:', idf_aft.shape)

idf_bef[['cusip8', 'AnnouncementDate']].to_csv(output + 'eventus_input_Q3_before.txt', sep=' ', index=False, header=False)
idf_aft[['cusip8', 'AnnouncementDate']].to_csv(output + 'eventus_input_Q3_after.txt', sep=' ', index=False, header=False)

Before shape: (45, 13) 
After shape: (7, 13)


In [54]:
parser(
    open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q3_before.txt', 'r').read(),
    **eventus_formatter['Abnormal']
)

,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",16,0.86%,1.97%,10:6,0.151,0.4399,1.854,0.0318,1.154,0.1243
1,"(-1,0)",16,0.71%,1.91%,11:5>,0.153,0.4391,2.197,0.0140,1.654,0.0491
2,"(0,+1)",16,0.10%,1.23%,10:6,0.021,0.4917,1.414,0.0787,1.154,0.1243
3,"(-5,+5)",16,0.63%,0.83%,10:6,0.058,0.4770,0.406,0.3424,1.154,0.1243
4,"(-10,+10)",16,-2.19%,-2.66%,7:9,-0.145,0.4422,-0.944,0.1726,-0.347,0.3641
5,"(-1,+1)",16,0.61%,1.78%,10:6,0.108,0.4570,1.672,0.0473,1.254,0.1048
6,"(-1,0)",16,0.60%,1.95%,11:5>,0.129,0.4487,2.248,0.0123,1.755,0.0396
7,"(0,+1)",16,-0.05%,0.96%,11:5>,-0.011,0.4957,1.110,0.1335,1.755,0.0396
8,"(-5,+5)",16,0.47%,0.89%,8:8,0.043,0.4827,0.438,0.3308,0.252,0.4003
9,"(-10,+10)",16,-3.00%,-2.91%,5:11,-0.200,0.4209,-1.035,0.1502,-1.250,0.1056


In [55]:
parser(
    open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q3_after.txt', 'r').read(),
    **eventus_formatter['Abnormal']
)

,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",4,3.37%,0.87%,3:1,0.711,0.2385,0.361,0.3592,1.109,0.1338
1,"(-1,0)",4,1.07%,1.23%,1:3,0.277,0.3908,0.624,0.2664,-0.894,0.1856
2,"(0,+1)",4,3.18%,1.16%,3:1,0.822,0.2055,0.584,0.2796,1.109,0.1338
3,"(-5,+5)",4,-3.68%,-3.16%,1:3,-0.405,0.3427,-0.682,0.2477,-0.894,0.1856
4,"(-10,+10)",4,8.40%,9.64%,4:0>,0.670,0.2514,1.504,0.0663,2.110,0.0174
5,"(-1,+1)",4,2.13%,0.25%,3:1,0.484,0.3143,0.102,0.4592,1.146,0.1259
6,"(-1,0)",4,0.97%,0.81%,3:1,0.270,0.3938,0.408,0.3417,1.146,0.1259
7,"(0,+1)",4,1.85%,0.43%,3:1,0.515,0.3034,0.216,0.4147,1.146,0.1259
8,"(-5,+5)",4,-7.48%,-4.14%,1:3,-0.887,0.1876,-0.892,0.1861,-0.859,0.1951
9,"(-10,+10)",4,-4.56%,3.41%,2:2,-0.391,0.3479,0.532,0.2975,0.143,0.4431


# 4. Shareholder reaction to repeal


In [59]:
board = pd.read_csv('./Data/Input/BoardEX/employment.csv', usecols=['DirectorName', 'CompanyName', 'BrdPosition', 'DirectorID', 'CompanyID', 'DateStartRole', 'DateEndRole']).drop_duplicates()\
          .query('BrdPosition in ["Yes", "Outside", "Inside"] and DateStartRole != "N" and DateEndRole != "N"')\
          .merge(
              pd.read_csv('./Data/Input/BoardEX/company.csv', usecols=['CIKCode', 'BoardID']).drop_duplicates(),
              how = 'left',
              left_on = 'CompanyID',
              right_on = 'BoardID'
          )\
          .merge(
              pd.read_csv('./Data/Input/BoardEX/individual.csv', usecols=['DirectorID', 'Gender']).drop_duplicates(),
              how = 'left',
              on = 'DirectorID'
          )\
          .rename({'CIKCode': 'CIK'}, axis=1)

          
board['DateStartRole'] = board['DateStartRole'].str.replace('-', '')
board['DateEndRole'] = board['DateEndRole'].str.replace('-', '')

board_at_event = board.query(f'DateStartRole <= "{event_date}" and ("{event_date}" <= DateEndRole or DateEndRole == "C") and CIK in [{",".join(ca["CIK"].astype(str))}]').copy()

board_composition = board_at_event.groupby(['CompanyID', 'CIK']).agg(
    n_dir = ('DirectorID', 'size'),
    n_F = ('Gender', lambda x: sum(x == 'F')),
    n_M = ('Gender', lambda x: sum(x == 'M')),
).reset_index().query('n_dir > 4')

board_composition['violation'] = 0
board_composition.loc[(board_composition['n_dir'] == 5) & (board_composition['n_F'] < 2), 'violation'] = 1
board_composition.loc[(board_composition['n_dir'] > 5) & (board_composition['n_F'] < 3), 'violation'] = 1

board_composition['no_women'] = 1
board_composition.loc[(board_composition['n_F'] > 0), 'no_women'] = 0

iss_composition = iss.merge(board_composition, how='left', on='CIK').dropna(subset=['violation'])
iss_composition['before_appeal'] = iss_composition['MeetingDate'].str.replace('-', '') < event_date

#   1. support for violation and non-violation firnms
display(iss_composition.groupby(['before_appeal', 'violation'])[['excess_support', 'support']].agg(['mean', 'size']))
display(iss_composition.groupby(['before_appeal', 'no_women'])[['excess_support', 'support']].agg(['mean', 'size']))
display(iss_composition.groupby(['before_appeal', 'no_women', 'violation'])[['excess_support', 'support']].agg(['mean', 'size']))

#   2. generate eventus input files
eventus_violation = board_composition.merge(ca_linked[['CIK', 'cusip8']], how='left', on='CIK')\
                                    .query('cusip8 == cusip8')\
                                    .assign(date = event_date)
                                    
eventus_violation.query('violation == 1')[['cusip8', 'date']].to_csv(output + 'eventus_input_Q4_violation.txt', sep=' ', index=False, header=False)
eventus_violation.query('violation == 0')[['cusip8', 'date']].to_csv(output + 'eventus_input_Q4_noviolation.txt', sep=' ', index=False, header=False)
eventus_violation.query('no_women == 0')[['cusip8', 'date']].to_csv(output + 'eventus_input_Q4_women.txt', sep=' ', index=False, header=False)
eventus_violation.query('no_women == 1')[['cusip8', 'date']].to_csv(output + 'eventus_input_Q4_nowomen.txt', sep=' ', index=False, header=False)

#       show the abnormal returns
print('Firms in violation')
display(parser(open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q4_violation.txt', 'r').read(), **eventus_formatter['Abnormal']))
print('Firms not in violation')
display(parser(open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q4_noviolation.txt', 'r').read(), **eventus_formatter['Abnormal']))
print('Firms with no women')
display(parser(open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q4_nowomen.txt', 'r').read(), **eventus_formatter['Abnormal']))
print('Firms with at least one women')
display(parser(open('./Data/Output/eventus/vwex_mar_-50_50_252_OLS_Q4_women.txt', 'r').read(), **eventus_formatter['Abnormal']))

excess_support         support      
                                  mean  size      mean  size
before_appeal violation                                     
False         0.0         4.373417e-19  1779  0.928069  1779
              1.0        -2.660665e-18   462  0.882577   462
True          0.0         7.199890e-19   772  0.946103   772
              1.0        -7.501507e-18   111  0.890281   111

excess_support         support      
                                 mean  size      mean  size
before_appeal no_women                                     
False         0.0       -2.756761e-19  2219  0.919508  2219
              1.0        5.286776e-18    22  0.836184    22
True          0.0       -3.146891e-19   883  0.939086   883

excess_support         support      
                                           mean  size      mean  size
before_appeal no_women violation                                     
False         0.0      0.0         4.373417e-19  1779  0.928069  1779
                       1.0        -3.295182e-18   440  0.884896   440
              1.0      1.0         5.286776e-18    22  0.836184    22
True          0.0      0.0         7.199890e-19   772  0.946103   772
                       1.0        -7.501507e-18   111  0.890281   111

Firms in violation


,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",206,6.62%,4.33%,145:61>>>,2.591,0.0048,9.929,<.0001,7.182,<.0001
1,"(-1,0)",206,4.53%,2.51%,137:69>>>,2.173,0.0149,7.057,<.0001,6.063,<.0001
2,"(0,+1)",206,1.93%,1.58%,127:79>>>,0.924,0.1776,4.432,<.0001,4.664,<.0001
3,"(-5,+5)",206,1.77%,0.70%,117:89>>>,0.362,0.3588,0.838,0.2011,3.265,0.0006
4,"(-10,+10)",206,-2.56%,-3.45%,92:114,-0.379,0.3525,-2.996,0.0014,-0.234,0.4077
5,"(-1,+1)",206,5.17%,3.32%,122:84>>>,2.115,0.0172,7.615,<.0001,3.267,0.0005
6,"(-1,0)",206,3.86%,2.05%,123:83>>>,1.937,0.0264,5.771,<.0001,3.407,0.0003
7,"(0,+1)",206,1.57%,1.34%,115:91>,0.786,0.2158,3.771,<.0001,2.291,0.0110
8,"(-5,+5)",206,5.37%,3.41%,136:70>>>,1.147,0.1258,4.086,<.0001,5.220,<.0001
9,"(-10,+10)",206,1.42%,-0.36%,107:99,0.219,0.4133,-0.309,0.3786,1.175,0.1200


...

,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",543,2.79%,1.62%,341:202>>>,1.612,0.0535,8.688,<.0001,7.293,<.0001
1,"(-1,0)",543,1.34%,0.66%,300:243>>>,0.950,0.1711,4.337,<.0001,3.769,<.0001
2,"(0,+1)",543,1.01%,0.62%,299:244>>>,0.716,0.2369,4.077,<.0001,3.683,0.0001
3,"(-5,+5)",543,0.95%,0.70%,269:274,0.286,0.3876,1.966,0.0247,1.104,0.1348
4,"(-10,+10)",543,-2.09%,-1.71%,255:288,-0.457,0.3240,-3.466,0.0003,-0.100,0.4603
5,"(-1,+1)",543,1.48%,0.90%,311:232>>>,0.923,0.1781,4.836,<.0001,3.939,<.0001
6,"(-1,0)",543,0.73%,0.32%,270:273,0.552,0.2903,2.112,0.0173,0.419,0.3375
7,"(0,+1)",543,0.66%,0.43%,290:253>,0.502,0.3079,2.806,0.0025,2.136,0.0163
8,"(-5,+5)",543,3.74%,2.25%,311:232>>>,1.214,0.1124,6.310,<.0001,3.939,<.0001
9,"(-10,+10)",543,0.79%,-0.11%,269:274,0.185,0.4267,-0.215,0.4147,0.333,0.3694


Firms with no women


,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",3,4.11%,5.08%,3:0>,0.957,0.1692,1.577,0.0574,1.788,0.0369
1,"(-1,0)",3,2.65%,3.99%,3:0>,0.756,0.2249,1.515,0.0649,1.788,0.0369
2,"(0,+1)",3,1.00%,0.62%,1:2,0.286,0.3874,0.235,0.4070,-0.523,0.3006
3,"(-5,+5)",3,-2.80%,0.78%,1:2,-0.341,0.3666,0.127,0.4496,-0.523,0.3006
4,"(-10,+10)",3,-2.00%,-1.24%,1:2,-0.176,0.4300,-0.146,0.4420,-0.523,0.3006
5,"(-1,+1)",3,4.96%,6.36%,3:0>,1.157,0.1236,1.974,0.0242,1.723,0.0425
6,"(-1,0)",3,3.16%,4.70%,2:1,0.903,0.1833,1.787,0.0370,0.568,0.2850
7,"(0,+1)",3,1.46%,1.19%,3:0>,0.416,0.3385,0.453,0.3251,1.723,0.0425
8,"(-5,+5)",3,-1.34%,1.38%,1:2,-0.163,0.4354,0.224,0.4113,-0.587,0.2788
9,"(-10,+10)",3,1.34%,1.23%,1:2,0.118,0.4532,0.144,0.4426,-0.587,0.2788


Firms with at least one women


,Days,N,Return,CAAR,Negative,CDA t,p-value,Z,p-value,Sign Z,p-value
0,"(-1,+1)",746,3.84%,2.17%,483:263>>>,2.010,0.0222,12.530,<.0001,9.879,<.0001
1,"(-1,0)",746,2.22%,1.03%,434:312>>>,1.421,0.0777,7.312,<.0001,6.284,<.0001
2,"(0,+1)",746,1.26%,0.82%,425:321>>>,0.811,0.2088,5.793,<.0001,5.623,<.0001
3,"(-5,+5)",746,1.19%,0.70%,385:361>>,0.325,0.3727,2.109,0.0175,2.688,0.0036
4,"(-10,+10)",746,-2.22%,-2.07%,346:400,-0.439,0.3304,-4.523,<.0001,-0.174,0.4308
5,"(-1,+1)",746,2.49%,1.39%,430:316>>>,1.390,0.0823,8.002,<.0001,4.968,<.0001
6,"(-1,0)",746,1.58%,0.67%,391:355>,1.083,0.1394,4.721,<.0001,2.111,0.0174
7,"(0,+1)",746,0.91%,0.61%,402:344>>,0.621,0.2674,4.347,<.0001,2.917,0.0018
8,"(-5,+5)",746,4.21%,2.49%,446:300>>>,1.228,0.1098,7.516,<.0001,6.140,<.0001
9,"(-10,+10)",746,0.96%,-0.16%,375:371,0.202,0.4198,-0.355,0.3611,0.939,0.1739
